In [ ]:
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind
import seaborn as sns
46
import os
import sys
wd = os.path.dirname(os.path.abspath(__file__))
os.chdir(wd)
input_path_main = 'result_df.csv'
df = pd.read_csv(input_path_main)
# Recoding weather_code
df['weather_good_bad'] = df.apply(lambda row: 1 if (row['weather_code'] < 20 and (16 <= row['temperature_2m_mean'] <= 26)) else 0, axis=1)
df['weather_rain'] = df['weather_code'].apply(lambda x: 1 if x >= 20 else 0)
# Compute Day of week
df['day_of_week'] = pd.to_datetime(df['date']).dt.dayofweek # Monday=0, Sunday=6
# Regression for H1 a
# Define interaction terms
df['interaction_weather_outdoorSeating'] = df['weather_good_bad'] * df['attributes.Out-doorSeating']
df['interaction_age_outdoorSeating'] = df['median_age'] * df['attributes.OutdoorSeat-ing']
df['interaction_weather_age'] = df['weather_good_bad'] * df['median_age']
df['interaction_age_weather_outdoorSeating'] = df['interaction_weather_outdoorSeat-ing'] * df['median_age']
# Define DV and IVs
X4A = df[['attributes.BikeParking', 'attributes.BusinessParking.Garage', 'attrib-utes.BusinessParking.Lot', 'attributes.BusinessParking.Street', 'attributes.BusinessPark-ing.Valet', 'attributes.BusinessParking.Validated', 'attributes.HappyHour', 'attrib-utes.HasTV', 'attributes.OutdoorSeating', 'attributes.RestaurantsReservations', 'attrib-utes.RestaurantsTableService', 'daily_average_rating', 'daily_review_count', 'day_of_week', 'median_age', 'precipitation_sum', 'scaled_capped_business_proximity', 'sunshine_duration', 'temperature_2m_mean', 'weather_good_bad', 'interac-tion_weather_outdoorSeating', 'interaction_age_outdoorSeating', 'interac-tion_weather_age', 'interaction_age_weather_outdoorSeating']]
pd.set_option('display.max_columns', None)
47
print(X4A.describe())
Y4A = df[['daily_checkin_count']]
print(Y4A.describe())
# Adding constant (intercept) to X
X4A= sm.add_constant(X4A)
# Fit the model
model4A = sm.OLS(Y4A, X4A).fit()
# Get the summary
print(model4A.summary())
# Regression for H2
# Define interaction terms
df['interaction_weather_parkingGarage'] = df['weather_rain'] * df['attributes.Business-Parking.Garage']
df['interaction_age_parkingGarage'] = df['median_age'] * df['attributes.BusinessPark-ing.Garage']
df['interaction_rainWeather_age'] = df['weather_rain'] * df['median_age']
df['interaction_age_rainWeahter_parkingGarage'] = df['interaction_weather_parking-Garage'] * df['median_age']
# Define DV and IVs
X4B = df[['attributes.BikeParking', 'attributes.BusinessParking.Garage', 'attrib-utes.BusinessParking.Lot', 'attributes.BusinessParking.Street', 'attributes.BusinessPark-ing.Valet', 'attributes.BusinessParking.Validated', 'attributes.HappyHour', 'attrib-utes.HasTV', 'attributes.OutdoorSeating', 'attributes.RestaurantsReservations', 'attrib-utes.RestaurantsTableService', 'daily_average_rating', 'daily_review_count', 'day_of_week', 'median_age', 'precipitation_sum', 'scaled_capped_business_proximity', 'sunshine_duration', 'temperature_2m_mean', 'weather_good_bad', 'interac-tion_weather_parkingGarage', 'interaction_age_parkingGarage', 'interaction_rain-Weather_age', 'interaction_age_rainWeahter_parkingGarage']]
Y4B = df[['daily_checkin_count']]
48
# Adding constant (intercept) to X
X4B = sm.add_constant(X4B)
# Fit the model
model4B = sm.OLS(Y4B, X4B).fit()
# Get the summary
print(model4B.summary())
sys.exit()
# Visualization
correlation_matrix = X.corr()
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=.5)
plt.show()
grouped_data = df.groupby(['weather_good_bad', 'attributes.OutdoorSeating', 'interac-tion_weather_outdoorSeating']).agg({'daily_checkin_count': 'mean'}).reset_index()
plt.figure(figsize=(8, 6))
sns.barplot(x='interaction_weather_outdoorSeating', y='daily_checkin_count', hue='weather_good_bad', data=grouped_data)
plt.title('Mean of Y for each combination of X1 and X2')
plt.xlabel('Interaction of X1 and X2')
plt.ylabel('Mean of Y')
plt.show()
Analysis Paycaret interpretation


In [ ]:
#This script utilizes PyCaret to apply ML Models onto our data. Regression Models are applied to predict the daily frequency of check ins at restaurants.

import pandas as pd
import os
import time
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from pycaret.regression import *
from pycaret.regression import pull
import sys
start_time = time.time()
# set working directory to where python file is located
wd = os.path.dirname(os.path.abspath(__file__))
os.chdir(wd)
input_path_main = 'result_df.csv'
df = pd.read_csv(input_path_main)
# Recoding weather_code
df['weather_good_bad'] = df.apply(lambda row: 1 if (row['weather_code'] < 20 and (16 <= row['temperature_2m_mean'] <= 26)) else 0, axis=1)
# New Feature: Day of the week
df['day_of_week'] = pd.to_datetime(df['date']).dt.dayofweek # Monday=0, Sunday=6
#print(list(df))
#sys.exit()
#########################
#
50
# PyCaret
#
#########################
# Create PySpark Session
#spark = SparkSession.builder.getOrCreate()
# Columns to ignore
ignore_columns = ['address', 'attributes.Alcohol', 'attributes.Alcohol.Beer_and_Wine', 'attributes.Alcohol.Full_Bar', 'attributes.Alcohol.None', 'attributes.BusinessParking', 'at-tributes.DogsAllowed', 'attributes.DriveThru', 'attributes.GoodForKids', 'attributes.Res-taurantsDelivery', 'attributes.RestaurantsTakeOut', 'business_id', 'business_proximity', 'capped_business_proximity', 'categories', 'checkin_dates', 'checkins_dur-ing_timeframe', 'city', 'date', 'first_checkin', 'is_open', 'last_checkin', 'latitude', 'longi-tude', 'name', 'postal_code', 'rain_sum', 'review_count', 'snowfall_sum', 'stars', 'state', 'temperature_2m_max', 'temperature_2m_min', 'weather_code']
# Setup the regression model with the chosen features and target
exp_reg = setup(data=df,
target='daily_checkin_count',
ignore_features=ignore_columns,
session_id=123,
use_gpu=True,
n_jobs=-1)
# Make predictions with the regression model
lightgbm = create_model('lightgbm')
from IPython.display import display
interpret_model(lightgbm)
interpret_model(lightgbm, plot='correlation')
shap_reason = interpret_model(lightgbm, plot='reason', observation=10) # Ensure you specify an observation index
display(shap_reason)
print('--- Runtime: %s seconds ---' % (time.time() - start_time))
51
'''
This script utilizes PyCaret to apply ML Models onto our data. Regression Models are applied to predict the daily frequency of check ins at restaurants.
'''
import pandas as pd
import os
import time
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from pycaret.regression import *
from pycaret.regression import pull
start_time = time.time()
# set working directory to where python file is located
wd = os.path.dirname(os.path.abspath(__file__))
os.chdir(wd)
input_path_main = 'result_df.csv'
df = pd.read_csv(input_path_main)
# Recoding weather_code
df['weather_good_bad'] = df.apply(lambda row: 1 if (row['weather_code'] < 20 and (16 <= row['temperature_2m_mean'] <= 26)) else 0, axis=1)
# New Feature: Day of the week
df['day_of_week'] = pd.to_datetime(df['date']).dt.dayofweek # Monday=0, Sunday=6


In [ ]:
# Correlation Matrix

# Calculate correlation matrix for a subset of variables that are relevant
correlation_vars = ['attributes.Alcohol.Full_Bar', 'attributes.Alcohol.Beer_and_Wine', 'attributes.Alcohol.None', 'attributes.BikeParking', 'attributes.OutdoorSeating', 'attrib-utes.BusinessParking.Garage', 'attributes.BusinessParking.Street', 'attributes.Business-Parking.Validated', 'attributes.BusinessParking.Lot', 'attributes.BusinessParking.Valet', 'attributes.RestaurantsTakeOut', 'attributes.RestaurantsDelivery', 'attributes.HasTV', 'at-tributes.DriveThru', 'attributes.HappyHour', 'attributes.DogsAllowed', 'attributes.Good-ForKids', 'attributes.RestaurantsReservations', 'attributes.RestaurantsTableService', 'scaled_capped_business_proximity', 'checkins_during_timeframe', 'daily_checkin_count', 'temperature_2m_max', 'temperature_2m_min', 'tempera-ture_2m_mean', 'sunshine_duration', 'precipitation_sum', 'rain_sum', 'snowfall_sum', 'daily_average_rating', 'daily_review_count', 'weather_good_bad', 'day_of_week', 'me-dian_age']
correlation_matrix = df[correlation_vars].corr()
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool)) # Create only one side of the triangle
# Plot the correlation matrix
plt.figure(figsize=(16, 16))
sns.heatmap(correlation_matrix, mask=mask, cmap='coolwarm', center=0,
square=True, linewidths=.5, cbar_kws={"shrink": .5})
plt.title('Correlation Matrix')
plt.savefig('correlation_matrix.png', dpi=300)
plt.show()
#########################
#
# PyCaret
#
#########################
# Columns to ignore
53
ignore_columns = ['address', 'attributes.Alcohol', 'attributes.Alcohol.Beer_and_Wine', 'attributes.Alcohol.Full_Bar', 'attributes.Alcohol.None', 'attributes.BusinessParking', 'at-tributes.DogsAllowed', 'attributes.DriveThru', 'attributes.GoodForKids', 'attributes.Res-taurantsDelivery', 'attributes.RestaurantsTakeOut', 'business_id', 'business_proximity', 'capped_business_proximity', 'categories', 'checkin_dates', 'checkins_dur-ing_timeframe', 'city', 'date', 'first_checkin', 'is_open', 'last_checkin', 'latitude', 'longi-tude', 'name', 'postal_code', 'rain_sum', 'review_count', 'snowfall_sum', 'stars', 'state', 'temperature_2m_max', 'temperature_2m_min', 'weather_code']
# Setup the regression model with the chosen features and target
exp_reg = setup(data=df,
target='daily_checkin_count',
ignore_features=ignore_columns,
session_id=123,
use_gpu=True,
n_jobs=-1)
# Train the regression model
best_model_reg = compare_models()
# Make predictions with the regression model
predictions_reg = create_model(best_model_reg)
automl()
time.sleep(20)
print("Prediction on Test:\n")
predict_model(best_model_reg)
plot_model(best_model_reg, plot='residuals', save=True)
plot_model(best_model_reg, plot='cooks', save=True)
plot_model(best_model_reg, plot='error', save=True)
plot_model(best_model_reg, plot='learning', save=True)
plot_model(best_model_reg, plot='vc', save=True)
plot_model(best_model_reg, plot='feature', save=True)
plot_model(best_model_reg, plot='feature_all', save=True)
try:
54
print(f"Feature Importances for {best_model_reg}:\n{best_model_reg.feature_im-portances_}")
except:
print(f"{best_model_reg} has no attribute 'feature_importance_")
try:
print(f"Model Coefficients for {best_model_reg}:\n{best_model_reg.coef_}")
except:
print(f"{best_model_reg} has no attribute 'coef_'")
print("Prediction on Test:\n")
predict_model(best_model_reg)
final_model = finalize_model(best_model_reg)
save_model(final_model, 'checkins_prediction_model')

# Tuning the model
print(f"--- Tuning the {best_model_reg} ---")
tuned_model = tune_model(best_model_reg)
automl()
time.sleep(10)
plot_model(tuned_model, plot='residuals', save=True)
plot_model(tuned_model, plot='cooks', save=True)
plot_model(tuned_model, plot='error', save=True)
plot_model(tuned_model, plot='learning', save=True)
plot_model(tuned_model, plot='vc', save=True)
plot_model(tuned_model, plot='feature', save=True)
plot_model(tuned_model, plot='feature_all', save=True)
try:
print(f"Feature Importances for {tuned_model}:\n{tuned_model.feature_impor-tances_}")
except:
print(f"{tuned_model} has no attribute 'feature_importance_")
try:
print(f"Model Coefficients for {tuned_model}:\n{tuned_model.coef_}")
55
except:
print(f"{tuned_model} has no attribute 'coef_'")
try:
plot_model(tuned_model, plot='parameter', save=True)
except:
print(f"Could not fetch parameters for {tuned_model}")
print("Prediction on Test:\n")
predict_model(tuned_model)
final_tuned_model = finalize_model(tuned_model)
save_model(final_tuned_model, 'checkins_prediction_model_tuned')
'''
print('--- Runtime: %s seconds ---' % (time.time() - start_time))